# Import Packages

In [5]:
library(opploansanalytics)
load.packages()
library(googlesheets4)

RPostgreSQL    RPostgres        RJDBC         httr     jsonlite googlesheets 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
   tidyverse        tidyr        dplyr    lubridate      stringr      ggplot2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
      ggpubr      cowplot        rJava        mailR     devtools     roxygen2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE

In [6]:
source('lde-raw-lead-validation.r')

In [7]:
admethod = getAdmethodList()
admethod %>% slice(1:20)

admethod,volume
<chr>,<int64>
LendingTree 4,8337
CreditKarma4,8181
Received Offer in Mail,8082
LenderEdge 4,7938
Even Financial 4,4639
Google,4595
Monevo,2943
GoogleBrand,2914
DigiohE,2476


# googlesheets4

In [8]:
sheets_auth()

Using an auto-discovered, cached token.
To suppress this message, modify your code or options to clearly consent to the use of a cached token.
See gargle's "Non-interactive auth" vignette for more details:
https://gargle.r-lib.org/articles/non-interactive-auth.html
The googlesheets4 package is using a cached token for jchang@opploans.com.


In [9]:
getPayloadDiagnostics = function (df, ss.url) {

    effective.sample.size =
        do.call(
            what = rbind,
            args = lenderedge.payload %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
        ) %>% 
        mutate(
            value.codes = value.codes %>% map(~ min(.x)) %>% unlist()
        ) %>% nrow()
    
    
    ##  FORMAT  ##
    do.call(
        what = rbind,
        args = lenderedge.payload %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
    ) %>% 
        mutate(
            value.codes = value.codes %>% map(~ min(.x)) %>% unlist()
        ) %>%
        group_by(
            key,
            value.expected.format = value.codes == 200,
            value.codes
        ) %>% 
        summarize(
            n = n(),
            p = n() / effective.sample.size
        ) %>% 
        ungroup() %>% 
        arrange(
            key,
            value.codes
        ) %T>%
        sheets_write(
            ss = ss.url,
            sheet = 'Value Data Types'
        )
    
    
    ##  MISSING KEYS  ##
    lenderedge.payload$keys.missing %>%
        unlist() %>%
        as.data.frame(
            stringsAsFactors = FALSE
        ) %>% 
        select(
            key = "."
        ) %>%
        group_by(
            key
        ) %>% 
        summarize(
            n = n(),
            p = n() / effective.sample.size
        ) %>% 
        ungroup() %>% 
        arrange(
            key
        ) %T>%
        sheets_write(
            ss = ss.url,
            sheet = 'Expected Keys Missing'
        )
    
    
    ##  EXTRA KEYS  ##
    lenderedge.payload$keys.extra %>%
        unlist() %>%
        as.data.frame(
            stringsAsFactors = FALSE
        ) %>% 
        select(
            key = "."
        ) %>%
        group_by(
            key
        ) %>% 
        summarize(
            n = n(),
            p = n() / effective.sample.size
        ) %>% 
        ungroup() %>% 
        arrange(
            key
        ) %T>%
        sheets_write(
            ss = ss.url,
            sheet = 'Unexpected Keys Present'
        )
    
    cat('Diagnostics Complete.')
    
}

### LenderEdge 4

In [ ]:
###  Other Analyses  ###

# lenderedge.reg = 'LenderEdge 4' %>% getRegInputs()

# payloadErrorRegCount(lenderedge.payload, lenderedge.reg)
# payloadErrorFields(lenderedge.payload, lenderedge.reg)

# payloadErrorSummary(lenderedge.payload)
# regFieldsSummary(lenderedge.payload, lenderedge.reg)

In [ ]:
# Add Missing / Extra Objects

In [10]:
lenderedge.payload = 'LenderEdge 4' %>% finalEvaluation(sample.size = 1000)

lenderedge.payload %>%
    getPayloadDiagnostics(
        'https://docs.google.com/spreadsheets/d/19ExrGhv1xnjFKhHa-G5ndpcOKWB6LVfwijVXqebeLCU/edit#gid=0'
    )

Auto-refreshing stale OAuth token.
Writing to "LenderEdge 4"
Writing to sheet "Value Data Types"
Writing to "LenderEdge 4"
Writing to sheet "Expected Keys Missing"
Writing to "LenderEdge 4"
Writing to sheet "Unexpected Keys Present"


Diagnostics Complete.

### Monevo

In [ ]:
monevo.payload = 'Monevo' %>% finalEvaluation(sample.size = 1000)
monevo.reg = 'Monevo' %>% getRegInputs()

In [ ]:
# payloadErrorRegCount(monevo.payload, monevo.reg)
# payloadErrorFields(monevo.payload, monevo.reg)

In [ ]:
payloadErrorSummary(monevo.payload)

In [ ]:
regFieldsSummary(monevo.payload, monevo.reg)

### Even Financial 4

In [ ]:
###  Even Financial 4
even.payload = 'Even Financial 4' %>% finalEvaluation(sample.size = 1000)
even.reg = 'Even Financial 4' %>% getRegInputs()

In [ ]:
# payloadErrorRegCount(even.payload, even.reg)
# payloadErrorFields(even.payload, even.reg)

In [ ]:
payloadErrorSummary(even.payload)

In [ ]:
regFieldsSummary(even.payload, even.reg)

In [ ]:
even.payload %>% slice(4) %>% .$validate.values

### LeadGroup

In [ ]:
###  LeadGroup
leadgroup.payload = 'LeadGroup' %>% finalEvaluation(sample.size = 1000)
leadgroup.reg = 'LeadGroup' %>% getRegInputs()

In [ ]:
leadgroup.payload %>%
    filter(
        validate.values %>% map(nrow) > 1
    ) %>% slice(98) %>% .$raw_lead %>% prettify()
#     filter(
# #     mutate(
# #         has.300 = 
#         validate.values %>% 
#             map(
#                 .f = function (x) {
#                     x %>%
#                         filter(
#                             value.codes %>% map(max) %>% > 200 &
#                             value.codes %>% map(max) < 900
#                         )
#                 }
#             ) %>% as.data.frame() %>% nrow() > 0
#     ) %>% 
#     slice(1)

In [ ]:
# payloadErrorRegCount(leadgroup.payload, leadgroup.reg)
# payloadErrorFields(leadgroup.payload, leadgroup.reg)

In [ ]:
payloadErrorSummary(leadgroup.payload)

In [ ]:
regFieldsSummary(leadgroup.payload, leadgroup.reg)

### Quin Street 4

In [ ]:
quinstreet.payload = 'Quin Street 4' %>% finalEvaluation(sample.size = 1000)
quinstreet.reg = 'Quin Street 4' %>% getRegInputs()

In [ ]:
# payloadErrorRegCount(quinstreet.payload, quinstreet.reg)
# payloadErrorFields(quinstreet.payload, quinstreet.reg)

In [ ]:
quinstreet.payload %>% filter(!is.na(name)) %>% slice(1)

In [ ]:
payloadErrorSummary(quinstreet.payload)

In [ ]:
regFieldsSummary(quinstreet.payload, quinstreet.reg)

### Avant

In [ ]:
avant.payload = 'Avant' %>% finalEvaluation(sample.size = 1000)
avant.reg = 'Avant' %>% getRegInputs()

In [ ]:
# payloadErrorRegCount(avant.payload, avant.reg)
# payloadErrorFields(avant.payload, avant.reg)

In [ ]:
payloadErrorSummary(avant.payload)

In [ ]:
regFieldsSummary(avant.payload, avant.reg)

### LeapTheory 4

In [ ]:
leaptheory.payload = 'LeapTheory 4' %>% finalEvaluation(sample.size = 1000)
leaptheory.reg = 'LeapTheory 4' %>% getRegInputs()

In [ ]:
# payloadErrorRegCount(leaptheory.payload, leaptheory.reg)
# payloadErrorFields(leaptheory.payload, leaptheory.reg)

In [ ]:
payloadErrorSummary(leaptheory.payload)

In [ ]:
regFieldsSummary(leaptheory.payload, leaptheory.reg)

### PersonalLoans.com 4

In [ ]:
personalloans.payload = 'PersonalLoans.com 4' %>% finalEvaluation(sample.size = 1000)
personalloans.reg = 'PersonalLoans.com 4' %>% getRegInputs()

In [ ]:
# payloadErrorRegCount(personalloans.payload, personalloans.reg)
# payloadErrorFields(personalloans.payload, personalloans.reg)

In [ ]:
payloadErrorSummary(personalloans.payload)

In [ ]:
regFieldsSummary(personalloans.payload, personalloans.reg)